### Notebook to build a Gradient Boosting Machine (GBM) classifier for infected and bystander cells 

- **Developed by**: Carlos Talavera-López Ph.D
- **Würzburg Institute for Systems Immunology & Faculty of Medicine, Julius-Maximilian-Universität Würzburg**
- v230813

### Import required modules

In [1]:
import anndata
import numpy as np
import pandas as pd
import scanpy as sc
import xgboost as xgb
import seaborn as sns
from pywaffle import Waffle
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

### Set up working environment

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'RdPu', dpi_save = 300, vector_friendly = True, format = 'svg')

### Read in Healthy-CTRL dataset

In [ ]:
adata = sc.read_h5ad('../data/Marburg_cell_states_locked_scANVI_ctl230813.raw.h5ad') 
adata

### Make a density plot for all viral counts in each group

In [ ]:
viral_genes = [gene for gene in adata.var_names if 'NC_' in gene]
adata.obs['viral_counts'] = np.sum(adata[:, viral_genes].X, axis = 1)

In [ ]:
plt.figure(figsize = (12, 6))
sns.kdeplot(data = adata.obs, x = 'viral_counts', hue = 'group', common_norm = False, fill = True)
plt.title('Density of Viral Transcript Counts per Group')
plt.xlabel('Total Viral Transcript Counts')
plt.ylabel('Density')
plt.xlim(0, 6500)
plt.show()

In [ ]:
group_means = adata.obs.groupby('group')['viral_counts'].mean()

# Plotting the barplot
plt.figure(figsize = (6, 8))
sns.barplot(x = group_means.index, y = group_means.values, palette = 'Dark2')

plt.xlabel('Group')
plt.ylabel('Average Viral Transcript Counts')
plt.title('Average Viral Transcripts per Group')
plt.tight_layout()
plt.show()

### Create labels for infected, non-infected and bystander/unknown

In [ ]:
def classify_cells(row):
    iav_genes = adata.var_names[adata.var_names.str.startswith('NC_')]
    iav_counts = row[adata.var_names.isin(iav_genes)]
    
    total_iav_count = iav_counts.sum()
    
    num_iav_genes_expressed = sum(iav_counts > 0)

    if num_iav_genes_expressed == len(iav_genes) and total_iav_count > 300:
        return 'infected'
    elif num_iav_genes_expressed <= 2 and total_iav_count < 300:
        return 'bystander'
    else:
        return 'uninfected'

In [ ]:
adata.obs['infection_status'] = [classify_cells(row) for row in adata.X.toarray()]

### Split data for training

In [ ]:
X = adata.X
y = adata.obs['infection_status'].values

label_encoder = LabelEncoder()
y_int = label_encoder.fit_transform(y)

X_train, X_test, y_train_int, y_test_int = train_test_split(X, y_int, test_size = 0.2, random_state = 1712)

In [ ]:
adata.obs['infection_status'].value_counts()

### Train GBM classifier

In [ ]:
dtrain = xgb.DMatrix(X_train, label = y_train_int)
dtest = xgb.DMatrix(X_test, label = y_test_int)

In [ ]:
# Parameters for XGBoost
param = {
    'max_depth': 6,
    'objective': 'multi:softprob',  # output probabilities
    'num_class': 3
}
num_round = 20

In [ ]:
watchlist = [(dtrain, 'train'), (dtest, 'test')]
bst = xgb.train(param, dtrain, num_round, evals=[(dtest, 'eval'), (dtrain, 'train')], verbose_eval=True)
preds = bst.predict(dtest)

In [ ]:
preds_prob = bst.predict(dtest)

# Convert the labels to a binary format for ROC curve
y_test_bin = label_binarize(y_test_int, classes = [0, 1, 2])

# Compute ROC curve for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], preds_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curve
colors = ['blue', 'red', 'green']
classes = ['infected', 'bystander', 'uninfected']
for i, color in zip(range(3), colors):
    plt.plot(fpr[i], tpr[i], color = color, lw = 2,
             label = '{0} (area = {1:0.2f})'.format(classes[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve for cell classification')
plt.legend(loc = "lower right")
plt.show()


In [ ]:
evals_result = {}
bst = xgb.train(param, dtrain, num_round, evals=[(dtest, 'eval'), (dtrain, 'train')],
                evals_result = evals_result, verbose_eval = True)

In [ ]:
print(evals_result['train'].keys())

In [ ]:
epochs = len(evals_result['train']['mlogloss'])
x_axis = range(0, epochs)

fig, ax = plt.subplots(figsize = (5, 5))
ax.plot(x_axis, evals_result['train']['mlogloss'], label = 'Train')
ax.plot(x_axis, evals_result['eval']['mlogloss'], label = 'Test') 
ax.legend()
plt.ylabel('Multi Class Log Loss')
plt.title('XGBoost Multi Class Log Loss')
plt.show()

### Visualise gene importance

In [ ]:
importance = bst.get_score(importance_type = 'weight')
sorted_importance = sorted(importance.items(), key = lambda x: x[1], reverse = True)

In [ ]:
gene_names = adata.var_names.tolist()
sorted_importance_with_names = [(gene_names[int(key[1:])], value) for key, value in sorted_importance]

In [ ]:
N = 25  # number of top features to display, change as needed
top_genes = sorted_importance_with_names[:N]
names, values = zip(*top_genes)

plt.figure(figsize = (8, 8))
plt.barh(names, values)
plt.gca().invert_yaxis()  # to display the most important gene at the top
plt.xlabel('Importance Score')
plt.title('Top {} Gene Importance'.format(N))
plt.show()


In [ ]:
sc.pl.umap(adata, frameon = False, color = ['group', 'disease', 'infection', 'infection_status', 'cell_states'], size = 1, legend_fontsize = 5, ncols = 4)

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['infection_status'], groups = ['bystander'] ,size = 1, legend_fontsize = 5, ncols = 4)

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['infection_status'], groups = ['infected'] ,size = 1, legend_fontsize = 5, ncols = 4)

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['infection_status'], groups = ['uninfected'] ,size = 1, legend_fontsize = 5, ncols = 4)

In [ ]:
df = adata.obs.groupby(['group', 'infection_status']).size().reset_index(name = 'counts')

grouped = df.groupby('group')['counts'].apply(lambda x: x / x.sum() * 100)
grouped = grouped.reset_index()

df['proportions'] = grouped['counts']
df['waffle_counts'] = (df['proportions'] * 10).astype(int)

In [ ]:
for group in df['group'].unique():
    temp_df = df[df['group'] == group]
    
    data = dict(zip(temp_df['infection_status'], temp_df['waffle_counts']))
    fig = plt.figure(
        FigureClass = Waffle, 
        rows = 8, 
        values = data, 
        title = {'label': f'Group {group}', 'loc': 'left'},
        labels = [f"{k} ({v}%)" for k, v in zip(temp_df['infection_status'], temp_df['proportions'].round(2))],
        #legend = {'loc': 'lower left', 'bbox_to_anchor': (0, -0.4), 'ncol': len(data), 'framealpha': 0},
        legend = {'loc': 'lower left', 'bbox_to_anchor': (0, -0.4), 'ncol': 12, 'framealpha': 0},
        figsize = (40, 4)
    )
    plt.show()

In [ ]:
data = pd.crosstab(adata.obs['group'], adata.obs['infection_status'])

ax1 = data.plot.barh(stacked = True, edgecolor = 'none', zorder = 3, figsize = (16,8), fontsize = 10, grid = False, width = 0.6)
horiz_offset = 1.0
vert_offset = 1.0
ax1.legend(bbox_to_anchor = (horiz_offset, vert_offset), ncol = 1)

for c in ax1.containers:
    
    labels = [f'{w:0.2f}' if (w := v.get_width()) > 0 else '' for v in c]
    ax1.bar_label(c, labels = labels, label_type = 'center', padding = 1, color = 'black', rotation = 270, fontsize = 6)